In [8]:
import pandas as pd
from sklearn import datasets
import numpy as np

#Taking it in as an np array
df = datasets.load_iris()

#converting it to a pandas dataframe
df = pd.DataFrame(data=np.c_[df['data'], df['target']],
                  columns= df['feature_names'] + ['target']) 

df


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [3]:
df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [4]:
df.target.value_counts()

0.0    50
1.0    50
2.0    50
Name: target, dtype: int64

In [9]:
#defining X and y
X = df.drop(['target'], axis=1)
y = df['target']


In [38]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from numpy import mean
from joblib import dump
from comet_ml import API, Experiment
import comet_ml
comet_ml.init(project_name="model_tracking")

model  = KNeighborsClassifier(n_neighbors=3)

def cross_val_eval(model, X, y):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)
    cv_scores = cross_val_score(model, X, y, cv = cv, scoring='accuracy', n_jobs=-1, error_score='raise')
    return cv_scores

cv_scores = cross_val_eval(model, X, y)
print(mean(cv_scores))

model.fit(X, y)
dump(model, 'model.joblib')

experiment = Experiment()
experiment.log_model("model1", "/home/mwaniki-new/Documents/Stacking/model.joblib")

experiment.end()


COMET INFO: Comet API key is valid
COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: sklearn. Metrics and hyperparameters can still be logged using Experiment.log_metrics() and Experiment.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


0.9533333333333334


COMET ERROR: Run will not be logged 
For more details, please refer to: https://www.comet.com/docs/python-sdk/warnings-errors/
Traceback (most recent call last):
  File "/home/mwaniki-new/.local/lib/python3.10/site-packages/comet_ml/experiment.py", line 975, in _start
    self.alive = self._setup_streamer()
  File "/home/mwaniki-new/.local/lib/python3.10/site-packages/comet_ml/_online.py", line 329, in _setup_streamer
    self._initialize_streamer(
  File "/home/mwaniki-new/.local/lib/python3.10/site-packages/comet_ml/_online.py", line 435, in _initialize_streamer
    self.ws_connection.wait_for_connection()
  File "/home/mwaniki-new/.local/lib/python3.10/site-packages/comet_ml/connection.py", line 1411, in wait_for_connection
    raise ValueError("Could not connect to server after multiple tries.")
ValueError: Could not connect to server after multiple tries.


In [37]:
api= API()
experiment = api.get("mwanikinjagi/model-tracking/intact_silo_3082")
experiment.register_model("model1")


CometRestApiException: POST https://www.comet.com/api/rest/v2/write/registry-model/item failed with status code 405: Registry Model Versions must be unique to promote experiment model.

In [30]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from numpy import std
from numpy import mean
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from comet_ml import API, Experiment

def model_dict():
    models = { 'knc' : KNeighborsClassifier(),
               'dtc' : DecisionTreeClassifier(),
               'xgc' : XGBClassifier(),
               'mlp' : MLPClassifier(max_iter=1000, random_state = 0),
               'lr'  : LogisticRegression(max_iter=500),
               'rf'  : RandomForestClassifier()
             }
    return models

def cross_val_eval(model, X, y):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)
    cv_scores = cross_val_score(model, X, y, cv = cv, scoring='accuracy', n_jobs=-1, error_score='raise')
    return cv_scores

models = model_dict()

#model evaluation
results, names = list(), list()
for name, model in models.items():
    scores = cross_val_eval(model, X, y)
    results.append(scores)
    names.append(names)
    print('%s %.5f (%.3f)' % (name, mean(scores), std(scores)))
    


knc 0.94667 (0.045)
dtc 0.94667 (0.016)
xgc 0.94667 (0.016)
mlp 0.96667 (0.030)
lr 0.96667 (0.030)
rf 0.95333 (0.040)


In [34]:

def get_models():
    models = dict()
    models['knc'] = KNeighborsClassifier()
    models['dtc'] = DecisionTreeClassifier()
    models['xgc'] = XGBClassifier()
    models['mlp'] = MLPClassifier(max_iter=1000, random_state = 0)  
    models['rfc'] = RandomForestClassifier()   
    return models

models = get_models()

model1 = MLPClassifier(max_iter=1000, random_state = 0)

model1.fit(X, y)
dump(model1, "model1.joblib")

results, names = list(), list()
for name, model in models.items():
    scores = cross_val_eval(model, X, y)
    results.append(scores)
    names.append(names)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))




>knc 0.947 (0.045)
>dtc 0.947 (0.016)
>xgc 0.947 (0.016)
>mlp 0.967 (0.030)
>rfc 0.953 (0.040)


In [40]:

experiment = Experiment()
api = API()
experiment.log_model("model1", "/home/mwaniki-new/Documents/Stacking/model1.joblib")
experiment.end()


COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: sklearn. Metrics and hyperparameters can still be logged using Experiment.log_metrics() and Experiment.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/home/mwaniki-new/Documents/Stacking' nor in any parent directory. You can override where Comet is looking for a Git Patch by setting the configuration `COMET_GIT_DIRECTORY`
COMET INFO: Experiment is live on comet.com https://www.comet.com/mwanikinjagi/model-tracking/8fc4efdf1aac4bf496fb732be9c45702

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.c

In [41]:
experiment = api.get("mwanikinjagi/model-tracking/intact_krill_7478")
experiment.register_model(model_name="model1", version="1.1.0")
experiment.end()

COMET INFO: Successfully registered 'model1', version '1.1.0' in workspace 'mwanikinjagi'
